<a href="https://colab.research.google.com/github/shreyasguha/Machine-Translation/blob/main/FinetuneLLM_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AI/Draco/train.csv')

In [ ]:
model_checkpoint = 'distilbert-base-uncased'

id2label = {0: "Bengali", 1: "Hindi", 2: "Punjabi", 3: "Tamil", 4: "Telugu"}
label2id = {"Bengali": 0, "Hindi": 1, "Punjabi": 2, "Tamil": 3, "Telugu": 4}

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=5, id2label=id2label, label2id=label2id
)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df.head()

,ben,guj,hin,kan,mal,ori,pan,tam,tel,urd,eng
0,sidny opera houser ashpashe thaka suijarlander...,sydney opera house aaspaasma yojayeli spardham...,sidney opera house kii prishthbhumi main switz...,sydney opera housein avarandalli aayojislagidd...,"sydney oppara hasiൻre parisarsliൽwech, switzeർ...",sidni opera houser pariveshtani madhyare anust...,sidney opera house de aale-duale sthit svissza...,sidney opera housein suzlil natant ant vilyatl...,sydney opera house parisarall jarigin i kridal...,"sidney opera house ke gurdonvah main kaayam, s...",Set in the surroundings of the Sydney Opera Ho...
1,con jivanu upar kaaj karbe sei anuyayi aigulo ...,sukshmajivanuona jooth par karaati asarne aadh...,jin sookshmaanuon ke samooh par ye kaam karti ...,ivugannu avu prenam biruv sookshmajivi gumpige...,badhikkunn sukshmajiviute waർggae atisthanmaki...,yeun bhootaanu goshthigudik upare eha kaam kar...,"inhan nuun anti- vistyle, anti- fungel, anti-p...",ews ent vaka nunnurigolin midhu vinapurikinden...,ivi prabhavan chupe sukshmajivas samuhaniki an...,inhen in jaraasam ke group ke lihaaz se jin pa...,"These are classified as anti-virals, anti-fung..."
2,bharter drut arth lenden vyavastha- upiai-ke a...,bharat bhugatan vyavasthao anaya adhikarakshet...,bharat kii sabase twarit bhugtan pranali - upi...,bharatad vegavad pavati vyavastheyad you.pi.i....,indiaille pettennu panamadaykanavunn nilevilul...,bharatar drut arth pradaan brivastha - eu.pi.a...,bharat de bhugtan karan wale shaylia da doosar...,indiavin yubis enepatum duridab panamseluttal ...,saguthunn bharat shighra chellimp vidhana - yu...,bhaarat ke adaegi ke nizaam ko digar dayera ak...,The possibility of linking India's payment sys...
3,fasal-poorvavarti evan fasal-paravarty caryakr...,paak poorveni ane paak pachhini pravrittio mat...,fasal kii kataai se pehle or uske baad die jan...,coylige munchin hagu coylen nantarad chatuvati...,villvetuppinumumpu villvetuppinusheavumulla pr...,amal-poorvavartti tatha amal-paravartti karyan...,fasal di kataai ton pihala ate bood vich us te...,aruvadiku mundaiy manllum bindaiy seyalbadugal...,"pantakotku mundu, taruvati panul koraku ichche...",kataai se kabel or kataai ke baad kii sargharm...,Loans for pre-harvest and post-harvest activit...
4,maldwar parikshar paddhati ekati drut paddhati...,gudamarg deetha thati tapas ek jhadpi padhdhat...,malashay-sambandhi pareekshan ek twarit vidhi ...,antah-gudnal pariksheyu kshipravad vidhanvagid...,computaർ samvidhana upyogitchull gudaparishodh...,malashai pariksha eka trit paddhati ate (jane ...,andakosh sambandi nirikhan ik jaladi on vaali ...,asana malakutal parichodanai woru viraivana va...,purishanalam dwara pariksh ock satvar vidhana ...,par-ractil maan ek tej tareeq car hai (ek taju...,The per-rectal examination is a quick method (...


In [ ]:
from datasets import load_dataset

dataset = load_dataset("shawhin/imdb-truncated")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dataset = pd.DataFrame(dataset["train"])

In [ ]:
dataset

,train
0,"{'label': 1, 'text': '. . . or type on a compu..."
1,"{'label': 1, 'text': 'During 1933 this film ha..."
2,"{'label': 0, 'text': 'Let me be clear. I've us..."
3,"{'label': 1, 'text': 'Carlos Mencia was excell..."
4,"{'label': 1, 'text': 'I was initially dubious ..."
...,...
995,"{'label': 0, 'text': 'I've seen this movie tod..."
996,"{'label': 0, 'text': 'The Incredible Melting M..."
997,"{'label': 0, 'text': 'With Adam Sandler.<br />..."
998,"{'label': 1, 'text': 'POSSIBLE SPOILER - In so..."


In [ ]:
df.head()

,ben,guj,hin,kan,mal,ori,pan,tam,tel,urd,eng
0,sidny opera houser ashpashe thaka suijarlander...,sydney opera house aaspaasma yojayeli spardham...,sidney opera house kii prishthbhumi main switz...,sydney opera housein avarandalli aayojislagidd...,"sydney oppara hasiൻre parisarsliൽwech, switzeർ...",sidni opera houser pariveshtani madhyare anust...,sidney opera house de aale-duale sthit svissza...,sidney opera housein suzlil natant ant vilyatl...,sydney opera house parisarall jarigin i kridal...,"sidney opera house ke gurdonvah main kaayam, s...",Set in the surroundings of the Sydney Opera Ho...
1,con jivanu upar kaaj karbe sei anuyayi aigulo ...,sukshmajivanuona jooth par karaati asarne aadh...,jin sookshmaanuon ke samooh par ye kaam karti ...,ivugannu avu prenam biruv sookshmajivi gumpige...,badhikkunn sukshmajiviute waർggae atisthanmaki...,yeun bhootaanu goshthigudik upare eha kaam kar...,"inhan nuun anti- vistyle, anti- fungel, anti-p...",ews ent vaka nunnurigolin midhu vinapurikinden...,ivi prabhavan chupe sukshmajivas samuhaniki an...,inhen in jaraasam ke group ke lihaaz se jin pa...,"These are classified as anti-virals, anti-fung..."
2,bharter drut arth lenden vyavastha- upiai-ke a...,bharat bhugatan vyavasthao anaya adhikarakshet...,bharat kii sabase twarit bhugtan pranali - upi...,bharatad vegavad pavati vyavastheyad you.pi.i....,indiaille pettennu panamadaykanavunn nilevilul...,bharatar drut arth pradaan brivastha - eu.pi.a...,bharat de bhugtan karan wale shaylia da doosar...,indiavin yubis enepatum duridab panamseluttal ...,saguthunn bharat shighra chellimp vidhana - yu...,bhaarat ke adaegi ke nizaam ko digar dayera ak...,The possibility of linking India's payment sys...
3,fasal-poorvavarti evan fasal-paravarty caryakr...,paak poorveni ane paak pachhini pravrittio mat...,fasal kii kataai se pehle or uske baad die jan...,coylige munchin hagu coylen nantarad chatuvati...,villvetuppinumumpu villvetuppinusheavumulla pr...,amal-poorvavartti tatha amal-paravartti karyan...,fasal di kataai ton pihala ate bood vich us te...,aruvadiku mundaiy manllum bindaiy seyalbadugal...,"pantakotku mundu, taruvati panul koraku ichche...",kataai se kabel or kataai ke baad kii sargharm...,Loans for pre-harvest and post-harvest activit...
4,maldwar parikshar paddhati ekati drut paddhati...,gudamarg deetha thati tapas ek jhadpi padhdhat...,malashay-sambandhi pareekshan ek twarit vidhi ...,antah-gudnal pariksheyu kshipravad vidhanvagid...,computaർ samvidhana upyogitchull gudaparishodh...,malashai pariksha eka trit paddhati ate (jane ...,andakosh sambandi nirikhan ik jaladi on vaali ...,asana malakutal parichodanai woru viraivana va...,purishanalam dwara pariksh ock satvar vidhana ...,par-ractil maan ek tej tareeq car hai (ek taju...,The per-rectal examination is a quick method (...
